# Finetune a BERT Text Classifier with LightningTrainer

This is an advanced example for LightningTrainer, which demonstrates how to use LightningTrainer with Dataset.

If you just want to quickly convert your existing PyTorch Lightning scripts into Ray AIR, you can refer to this starter example: Train a Pytorch Lightning Image Classifier.

Source: https://docs.ray.io/en/latest/train/examples/lightning/lightning_cola_advanced.html

In [1]:
import ray
import torch
import pytorch_lightning as pl
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset, load_metric
import numpy as np

## Pre-process CoLA Dataset

In [2]:
dataset = load_dataset("glue", "cola")
metric = load_metric("glue", "cola")

ray_datasets = ray.data.from_huggingface(dataset)

Generating train split:   0%|          | 0/8551 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1043 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1063 [00:00<?, ? examples/s]

/tmp/ipykernel_747132/2004070181.py:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("glue", "cola")


2023-09-06 23:45:53,584	WARNING read_api.py:1794 -- You provided a Huggingface DatasetDict which contains multiple datasets. The output of `from_huggingface` is a dictionary of Ray Datasets. To convert just a single Huggingface Dataset to a Ray Dataset, specify a split. For example, `ray.data.from_huggingface(my_dataset_dictionary['train'])`. Available splits are ['train', 'validation', 'test'].
2023-09-06 23:45:55,467	INFO worker.py:1612 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8266 


In [3]:
from ray.data.preprocessors import BatchMapper

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_sentence(batch):
    encoded_sent = tokenizer(
        batch["sentence"].tolist(),
        max_length=128,
        truncation=True,
        padding="max_length",
        return_tensors="pt",
    )
    batch["input_ids"] = encoded_sent["input_ids"].numpy()
    batch["attention_mask"] = encoded_sent["attention_mask"].numpy()
    batch["label"] = np.array(batch["label"])
    batch.pop("sentence")
    return batch


preprocessor = BatchMapper(tokenize_sentence, batch_format="numpy")

## Define a PyTorch Lightning Model

In [4]:
class SentimentModel(pl.LightningModule):
    def __init__(self, lr=2e-5, eps=1e-8):
        super().__init__()
        self.lr = lr
        self.eps = eps # epsilon
        self.num_classes = 2
        self.model = AutoModelForSequenceClassification.from_pretrained(
            "bert-base-cased", num_labels=self.num_classes
        )
        self.metric = load_metric("glue", "cola")
        self.predictions = []
        self.references = []

    def forward(self, batch):
        input_ids, attention_mask = batch["input_ids"], batch["attention_mask"]
        outputs = self.model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        return logits

    def training_step(self, batch, batch_idx):
        labels = batch["label"]
        logits = self.forward(batch)
        loss = F.cross_entropy(logits.view(-1, self.num_classes), labels)
        self.log("train_loss", loss)
        return loss

    def validation_step(self, batch, batch_idx):
        labels = batch["label"]
        logits = self.forward(batch)
        preds = torch.argmax(logits, dim=1)
        self.predictions.append(preds)
        self.references.append(labels)

    def on_validation_epoch_end(self):
        predictions = torch.concat(self.predictions).view(-1)
        references = torch.concat(self.references).view(-1)
        matthews_correlation = self.metric.compute(
            predictions=predictions, references=references
        )

        # self.metric.compute() returns a dictionary:
        # e.g. {"matthews_correlation": 0.53}
        self.log_dict(matthews_correlation, sync_dist=True)
        self.predictions.clear()
        self.references.clear()

    def configure_optimizers(self):
        return torch.optim.AdamW(self.parameters(), lr=self.lr, eps=self.eps)

## Configure your LightningTrainer

In [5]:
from ray.train.lightning import LightningTrainer, LightningConfigBuilder
from ray.air.config import RunConfig, ScalingConfig, CheckpointConfig

# Define the configs for LightningTrainer
lightning_config = (
    LightningConfigBuilder()
    .module(cls=SentimentModel, lr=1e-5, eps=1e-8)
    .trainer(max_epochs=5, accelerator="gpu")
    .checkpointing(save_on_train_epoch_end=False)
    .build()
)

In [6]:
# Save AIR checkpoints according to the performance on validation set
run_config = RunConfig(
    name="ptl-sent-classification",
    checkpoint_config=CheckpointConfig(
        num_to_keep=2,
        checkpoint_score_attribute="matthews_correlation",
        checkpoint_score_order="max",
    ),
)

# Scale the DDP training workload across 4 GPUs
# You can change this config based on your compute resources.
scaling_config = ScalingConfig(
    num_workers=1, use_gpu=True, resources_per_worker={"CPU": 10, "GPU": 1}
)

## Fine-tune the model with LightningTrainer

In [7]:
trainer = LightningTrainer(
    lightning_config=lightning_config,
    run_config=run_config,
    scaling_config=scaling_config,
    datasets={"train": ray_datasets["train"], "val": ray_datasets["validation"]},
    datasets_iter_config={"batch_size": 16},
    preprocessor=preprocessor,
)
result = trainer.fit()

(LightningTrainer pid=768021) The `preprocessor` arg to Trainer is deprecated. Apply preprocessor transformations ahead of time by calling `preprocessor.transform(ds)`. Support for the preprocessor arg will be dropped in a future release.
(LightningTrainer pid=768021) Starting distributed worker processes: ['768112 (192.168.1.147)']
(RayTrainWorker pid=768112) Setting up process group for: env:// [rank=0, world_size=1]
(RayTrainWorker pid=768112) Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
(RayTrainWorker pid=768112) You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
(RayTrainWorker pid=768112) /tmp/ipykernel_747132/3457447719.py:10: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: 

(pid=768112) Running 0:   0%|          | 0/1 [00:00<?, ?it/s]

(pid=768112) - RandomizeBlockOrder: 0 active, 0 queued, 0.0 MiB objects, 0 output 1:   0%|          | 0/1 [00:…

Sanity Checking DataLoader 0:  50%|█████     | 1/2 [00:00<00:00,  2.68it/s]
Epoch 0: : 0it [00:00, ?it/s]                                              


(RayTrainWorker pid=768112) /home/dino/anaconda3/envs/ray/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:442: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
(RayTrainWorker pid=768112)   rank_zero_warn(
(RayTrainWorker pid=768112) Executing DAG InputDataBuffer[Input] -> TaskPoolMapOperator[MapBatches(BatchMapper._transform_numpy)] -> AllToAllOperator[RandomizeBlockOrder]
(RayTrainWorker pid=768112) Execution config: ExecutionOptions(resource_limits=ExecutionResources(cpu=None, gpu=None, object_store_memory=None), locality_with_output=False, preserve_order=False, actor_locality_enabled=True, verbose_progress=False)
(RayTrainWorker pid=768112) Tip: For detailed progress reporting, run `ray.data.DataContext.get_current().execution_o

...

(pid=768112) - RandomizeBlockOrder: 0 active, 0 queued, 0.0 MiB objects, 0 output 1:   0%|          | 0/1 [00:…

(RayTrainWorker pid=768112) 
Validation: 0it [00:00, ?it/s]12) 
Validation: 0it [00:00, ?it/s]12) 
Validation DataLoader 0: : 0it [00:00, ?it/s]
Validation DataLoader 0: : 1it [00:00, 99.27it/s]
Validation DataLoader 0: : 2it [00:00, 99.76it/s]
Validation DataLoader 0: : 3it [00:00, 101.74it/s]
(RayTrainWorker pid=768112) 
Validation DataLoader 0: : 4it [00:00, 26.38it/s] 
Validation DataLoader 0: : 5it [00:00, 31.09it/s]
Validation DataLoader 0: : 6it [00:00, 35.43it/s]
(RayTrainWorker pid=768112) 
Validation DataLoader 0: : 7it [00:00, 24.04it/s]
Validation DataLoader 0: : 8it [00:00, 26.61it/s]
Validation DataLoader 0: : 9it [00:00, 29.13it/s]
(RayTrainWorker pid=768112) 
Validation DataLoader 0: : 10it [00:00, 23.21it/s]
Validation DataLoader 0: : 11it [00:00, 25.02it/s]
Validation DataLoader 0: : 12it [00:00, 26.82it/s]
(RayTrainWorker pid=768112) 
Validation DataLoader 0: : 13it [00:00, 22.79it/s]
Validation DataLoader 0: : 14it [00:00, 24.18it/s]
(RayTrainWorker pid=768112) 
Val

(RayTrainWorker pid=768112) `Trainer.fit` stopped: `max_epochs=5` reached.


Epoch 4: : 535it [01:41,  5.29it/s, v_num=0]


2023-09-07 00:00:27,703	INFO tune.py:1148 -- Total run time: 564.58 seconds (564.56 seconds for the tuning loop).


In [8]:
result

Result(
  metrics={'_report_on': 'validation_end', 'train_loss': 0.08543746918439865, 'matthews_correlation': 0.5930452712523209, 'epoch': 4, 'step': 2675, 'should_checkpoint': True, 'done': True, 'trial_id': '9082c_00000', 'experiment_tag': '0'},
  path='/home/dino/ray_results/ptl-sent-classification/LightningTrainer_9082c_00000_0_2023-09-06_23-51-03',
  checkpoint=LightningCheckpoint(local_path=/home/dino/ray_results/ptl-sent-classification/LightningTrainer_9082c_00000_0_2023-09-06_23-51-03/checkpoint_000004)
)